# Using Streamlit with Ollama for sentiment Analysis

    In this section we will go over how we can pair ollama with streamlit to use NLP with ollama to perform sentiment analysis on text. 

## 1. Install necessary Packages
    !pip install ollama streamlit

In [36]:
!pip install ollama streamlit

In [37]:
# test ollama to make sure it works
import ollama

In [38]:
ollama.list()

{'models': [{'name': 'spam_Ollama:latest',
   'model': 'spam_Ollama:latest',
   'modified_at': '2024-12-05T16:18:29.756616099-06:00',
   'size': 807691078,
   'digest': 'a5aa4550ffc57110ae2c2c2ae3f1108eb8bfdc938d552f93de7bd8ad0ba10d07',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '1.2B',
    'quantization_level': 'Q4_K_M'}},
  {'name': 'llama3.2:latest',
   'model': 'llama3.2:latest',
   'modified_at': '2024-10-16T19:26:18.135739959-05:00',
   'size': 2019393189,
   'digest': 'a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '3.2B',
    'quantization_level': 'Q4_K_M'}}]}

In [39]:
## To chat with llma3.2
prompt={'role':'user', 'content':"With approximately 100 words explain what is large language model?"}
llm='llama3.2'
response = ollama.chat(model=llm, messages=[prompt])


print(response)

{'model': 'llama3.2', 'created_at': '2025-01-30T17:51:22.690776Z', 'message': {'role': 'assistant', 'content': "A large language model (LLM) is a type of artificial intelligence (AI) designed to process and understand human language. It's trained on vast amounts of text data, which enables it to learn patterns, relationships, and context. LLMs use complex algorithms to generate responses, answers, or text based on the input they receive. They can be used for tasks such as language translation, text summarization, question answering, and more. LLMs have become increasingly popular in areas like natural language processing (NLP), chatbots, and virtual assistants due to their ability to understand and respond to human-like language inputs."}, 'done_reason': 'stop', 'done': True, 'total_duration': 9845562292, 'load_duration': 816609125, 'prompt_eval_count': 37, 'prompt_eval_duration': 4087000000, 'eval_count': 127, 'eval_duration': 4937000000}


In [40]:
answer = response['message']['content']
print(answer)

A large language model (LLM) is a type of artificial intelligence (AI) designed to process and understand human language. It's trained on vast amounts of text data, which enables it to learn patterns, relationships, and context. LLMs use complex algorithms to generate responses, answers, or text based on the input they receive. They can be used for tasks such as language translation, text summarization, question answering, and more. LLMs have become increasingly popular in areas like natural language processing (NLP), chatbots, and virtual assistants due to their ability to understand and respond to human-like language inputs.


## 2. Create a simple application with streamlit as a foundation


In [52]:
%%writefile sentiment_app.py

import streamlit as st
import ollama

st.title('Ollama Sentiment Analysis')
st.caption('''
    This application is used to perform sentiment analysis 
    on input text data and will score the input as "Positive",
    "Negative", or "Neutral."
    ''')

llm='llama3.2'
user_input = str(st.chat_input('Place text for review here:'))

response=ollama.generate(model=llm, prompt="Respond only with 'Positive' or 'Negative' or 'Neutral' for the review:"+user_input)

if user_input:
    contents=response['response']
    st.write(contents)

Overwriting sentiment_app.py


In [56]:
!streamlit run sentiment_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8503
  Network URL: http://10.112.132.163:8503

^C
  Stopping...


In [54]:
%%writefile sentiment_app.py

import streamlit as st
import ollama
import pandas as pd
import plotly.express as px

# Sidebar for selecting application mode
page = st.sidebar.selectbox(
    "Choose a Mode",
    ["Ollama Chat", "Sentiment Analysis"]
)

if page == "Ollama Chat":
    st.title("Ollama Chat")
    st.caption("A simple chatbot using Ollama.")

    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []

    user_input = st.text_input("Enter your message:")
    
    if user_input:
        response = ollama.generate(model="llama3.2", prompt=user_input)
        if 'response' in response:
            st.session_state.chat_history.append(("You", user_input))
            st.session_state.chat_history.append(("Ollama", response['response']))
    
    for sender, msg in st.session_state.chat_history:
        st.write(f"**{sender}:** {msg}")


elif page == "Sentiment Analysis":
    st.title("Ollama Sentiment Analysis")
    st.caption(
        "Enter text to classify sentiment as 'Positive', 'Negative', or 'Neutral'.\n\n"
        "This works by using a prompt-based approach with Ollama, where the model is "
        "instructed to strictly respond with one of the three sentiment labels. "
        "By carefully crafting the prompt, we guide the model to provide structured "
        "and predictable responses, ensuring consistency in sentiment classification."
    )

    analysis_mode = st.radio("Choose Input Mode:", ["Single Review", "Batch Processing (CSV)"])

    if analysis_mode == "Single Review":
        user_input = st.chat_input("Place text for review here:")

        if user_input:
            sentiment_prompt = f"Classify the following review as 'Positive', 'Negative', or 'Neutral': {user_input}"
            response = ollama.generate(model="llama3.2", prompt=sentiment_prompt)

            if 'response' in response:
                st.write(f"**Sentiment:** {response['response']}")

    elif analysis_mode == "Batch Processing (CSV)":
        uploaded_file = st.file_uploader("Upload a CSV file with a column named 'review'", type=["csv"])

        if uploaded_file is not None:
            df = pd.read_csv(uploaded_file)

            if "review" not in df.columns:
                st.error("The uploaded CSV must have a column named 'review'.")
            else:
                st.write("Processing reviews... This may take some time.")

                sentiments = []
                for review in df["review"]:
                    sentiment_prompt = f"Classify the following review as 'Positive', 'Negative', or 'Neutral': {review}"
                    response = ollama.generate(model="llama3.2", prompt=sentiment_prompt)
                    sentiments.append(response.get("response", "Unknown"))

                df["Sentiment"] = sentiments

                # Display results
                st.dataframe(df)

                # Create pie chart
                sentiment_counts = df["Sentiment"].value_counts()
                fig = px.pie(
                    names=sentiment_counts.index,
                    values=sentiment_counts.values,
                    title="Sentiment Distribution",
                    color=sentiment_counts.index,
                    color_discrete_map={"Positive": "green", "Negative": "red", "Neutral": "gray"},
                )

                st.plotly_chart(fig)

Overwriting sentiment_app.py
